# Academic QA System with GraphRAG

- Author: [Yongdam Kim](https://github.com/dancing-with-coffee)
- Design: 
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

This tutorial demonstrates how to implement a QA system that better leverages paper (academic) content by using GraphRAG.

GraphRAG is a novel system introduced by Microsoft that utilizes a graph to extract both local and global information from text, providing more contextually rich answers.

However, Microsoft’s official GraphRAG implementation is not readily integrated with LangChain, making it difficult to use.

To solve this, we use `langchain-graphrag` which allows us to implement GraphRAG within LangChain.

In this tutorial, we’ll learn how to build a QA system for the latest AI papers using `langchain-graphrag`.

![GraphRAG](./assets/08-academicqasystem-graphrag-pipeline-.png)

> from arXiv- "From Local to Global-A Graph RAG Approach to Query-Focused Summarization"

### Table of Contents

### References

1. [langchain-graphrag github repo](https://github.com/ksachdeva/langchain-graphrag)
2. [GraphRAG paper](https://arxiv.org/pdf/2404.16130)
3. [GraphRAG Microsoft Official Research Blog post](https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data/)

----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials.
- You can check out the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages.
# 'langchain_opentutorial.package.install' is a helper function that installs
# the specified packages inside this environment.

from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain-graphrag",
        "langchain_chroma",
        "jq",
    ],
    verbose=False,
    upgrade=False,
)

You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [3]:
# Load environment variables from a .env file (e.g. OPENAI_API_KEY)
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Load arXiv PDFs

In this tutorial, we will use arXiv data. arXiv is an online archive for the latest research papers, all available in PDF format. There is an official GitHub repository containing all PDFs, but it is about 1TB in total size and can only be downloaded from AWS.
Thus, in this tutorial, we will selectively use a few PDF files instead.

- Link to the full dataset: https://github.com/mattbierbaum/arxiv-public-datasets

In [4]:
# Load the GraphRAG paper using PyPDFLoader.
# PyPDFLoader loads PDF content on a per-page basis.

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/2404.16130v1.pdf")
docs = loader.load()
print(f"Loaded {len(docs)} documents.")
print(docs[0].page_content)

Loaded 15 documents.
From Local to Global: A Graph RAG Approach to
Query-Focused Summarization
Darren Edge1†Ha Trinh1†Newman Cheng2Joshua Bradley2Alex Chao3
Apurva Mody3Steven Truitt2
Jonathan Larson1
1Microsoft Research
2Microsoft Strategic Missions and Technologies
3Microsoft Office of the CTO
{daedge,trinhha,newmancheng,joshbradley,achao,moapurva,steventruitt,jolarso }
@microsoft.com
†These authors contributed equally to this work
Abstract
The use of retrieval-augmented generation (RAG) to retrieve relevant informa-
tion from an external knowledge source enables large language models (LLMs)
to answer questions over private and/or previously unseen document collections.
However, RAG fails on global questions directed at an entire text corpus, such
as “What are the main themes in the dataset?”, since this is inherently a query-
focused summarization (QFS) task, rather than an explicit retrieval task. Prior
QFS methods, meanwhile, fail to scale to the quantities of text indexed by typi

## Text Chunking and Text Extracting

In this step, we perform **query routing** and **document evaluation**. These steps are crucial parts of **Adaptive RAG**, contributing to efficient information retrieval and generation.

- **Query Routing**: Analyzes the user’s query to route it to the appropriate information source.
- **Document Evaluation**: Evaluates the quality and relevance of the retrieved documents.

These steps support the core functions of **Adaptive RAG**, aiming to provide accurate and reliable information.

In [5]:
# Here we split the loaded documents into text chunks.
# 'RecursiveCharacterTextSplitter' is a commonly used utility in LangChain for
# chunking text with some overlap.

from langchain_core.documents import Document
from langchain_graphrag.indexing import TextUnitExtractor
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
text_unit_extractor = TextUnitExtractor(text_splitter=splitter)

# This runs the text splitting logic on the loaded PDF pages
df_text_units = text_unit_extractor.run(docs)
df_text_units

Processing documents ...: 100%|██████████| 15/15 [00:00<00:00, 612.87it/s]


,document_id,id,text_unit
0,c60db795-9f98-48b4-99f4-67ddd817dfe4,ddce20dc-9c4c-486f-bbd6-0b9e94bec550,From Local to Global: A Graph RAG Approach to\...
1,c60db795-9f98-48b4-99f4-67ddd817dfe4,fb96abaa-e4bd-4f49-8d0e-61a0969681fe,tion from an external knowledge source enables...
2,c60db795-9f98-48b4-99f4-67ddd817dfe4,5a2c9472-0248-4346-b9f8-db1c4cc4c6bb,RAG systems. To combine the strengths of these...
3,c60db795-9f98-48b4-99f4-67ddd817dfe4,23435e34-e9ef-40de-88ee-92788172e4c1,"question, each community summary is used to ge..."
4,c60db795-9f98-48b4-99f4-67ddd817dfe4,2d51a8c6-9a8f-4faf-9d2a-61e67394e66f,approaches is forthcoming at https://aka .ms/g...
...,...,...,...
117,342c63f4-0077-4a21-84fd-76c8ca6dfd3e,961f62d7-205c-4f60-8b5c-4d71275d4031,with chain-of-thought reasoning for knowledge-...
118,342c63f4-0077-4a21-84fd-76c8ca6dfd3e,2bd75839-3cca-4108-878c-26f25c9e2798,"Wang, Y ., Lipka, N., Rossi, R. A., Siu, A., Z..."
119,342c63f4-0077-4a21-84fd-76c8ca6dfd3e,8a6d70cb-b55e-4dd7-989c-d5f2e773b80b,Empirical Methods in Natural Language Processi...
120,648cb424-8581-443f-98ef-182b03395e0e,32299506-965b-41cb-9d6f-13818f55b9ec,"Yao, L., Peng, J., Mao, C., and Luo, Y . (2023..."


### Entity Relationship Extraction

GraphRAG extracts entities and relationships from the text chunks to automatically build a knowledge graph.

When constructing a Knowledge Graph, an LLM is used. In this tutorial, we use `gpt-4o-mini` for performance and cost reasons. The LLM uses a predefined prompt to extract entity and relationship information.

In [6]:
# This process can take about 20 minutes.
# EntityRelationshipExtractor uses a language model to identify entities & relationships in the text.

from langchain_graphrag.indexing.graph_generation import EntityRelationshipExtractor
from langchain_openai import ChatOpenAI

# We instantiate a ChatOpenAI with 'gpt-4o-mini' model.
er_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Build the default entity-relationship extractor.
extractor = EntityRelationshipExtractor.build_default(llm=er_llm)

# Invoke extractor on the text units (chunks)
text_unit_graphs = extractor.invoke(df_text_units)

Extracting entities and relationships ...: 100%|██████████| 122/122 [20:53<00:00, 10.28s/it]


In [7]:
# Display the graph information (nodes/edges) extracted from each chunk.
for index, g in enumerate(text_unit_graphs):
    print("---------------------------------")
    print(f"Graph: {index}")
    print(f"Number of nodes - {len(g.nodes)}")
    print(f"Number of edges - {len(g.edges)}")
    print(g.nodes())
    print(g.edges())
    print("---------------------------------")

---------------------------------
Graph: 0
Number of nodes - 11
Number of edges - 8
['DARREN EDGE', 'HA TRINH', 'NEWMAN CHENG', 'JOSHUA BRADLEY', 'ALEX CHAO', 'APURVA MODY', 'STEVEN TRUITT', 'JONATHAN LARSON', 'MICROSOFT RESEARCH', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES', 'MICROSOFT OFFICE OF THE CTO']
[('DARREN EDGE', 'MICROSOFT RESEARCH'), ('HA TRINH', 'MICROSOFT RESEARCH'), ('NEWMAN CHENG', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES'), ('JOSHUA BRADLEY', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES'), ('ALEX CHAO', 'MICROSOFT OFFICE OF THE CTO'), ('APURVA MODY', 'MICROSOFT OFFICE OF THE CTO'), ('STEVEN TRUITT', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES'), ('JONATHAN LARSON', 'MICROSOFT RESEARCH')]
---------------------------------
---------------------------------
Graph: 1
Number of nodes - 3
Number of edges - 2
['LARGE LANGUAGE MODELS', 'QUERY-FOCUSED SUMMARIZATION', 'TEXT CORPUS']
[('LARGE LANGUAGE MODELS', 'QUERY-FOCUSED SUMMARIZATION'), ('QUERY-FOCUSED SUMMARI

In [8]:
# Example: search for a specific extracted node
text_unit_graphs[0].nodes["MICROSOFT RESEARCH"]

{'type': 'ORGANIZATION',
 'description': ['Microsoft Research is a division of Microsoft focused on advanced research and development, and is involved in the paper on Query-Focused Summarization.'],
 'text_unit_ids': ['ddce20dc-9c4c-486f-bbd6-0b9e94bec550']}

In [9]:
# Example: check the relationship (edge) between two extracted entities
text_unit_graphs[0].edges[("MICROSOFT RESEARCH", "DARREN EDGE")]

{'weight': 1.0,
 'description': ['Darren Edge is a researcher at Microsoft Research, contributing to the paper on Query-Focused Summarization'],
 'text_unit_ids': ['ddce20dc-9c4c-486f-bbd6-0b9e94bec550']}

### Graph Generation

GraphRAG does not use all extracted entities and relationships individually; it merges them into a more comprehensive structure. We call this process Summarization.

Through element summarization, GraphRAG enhances search functionality by improving global context understanding.

In [10]:
# This process can take about 22 minutes.
# We merge all local graphs into a single consolidated graph, then run summarization.

from langchain_graphrag.indexing.graph_generation import (
    GraphsMerger,
    EntityRelationshipDescriptionSummarizer,
    GraphGenerator,
)

graphs_merger = GraphsMerger()

es_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

summarizer = EntityRelationshipDescriptionSummarizer.build_default(llm=es_llm)

graph_generator = GraphGenerator(
    er_extractor=extractor,
    graphs_merger=GraphsMerger(),
    er_description_summarizer=summarizer,
)

# Execute the graph generation.
graph = graph_generator.run(df_text_units)

Summarizing relationship descriptions: 100%|██████████| 728/728 [00:27<00:00, 26.44it/s]


In [11]:
# Check how many nodes and edges are in the final merged+summarized graph.
print(f"Number of nodes - {len(graph[0].nodes)}")
print(f"Number of edges - {len(graph[0].edges)}")

Number of nodes - 501
Number of edges - 728


## Graph Index Build

- We run all steps from Text Chunking to Community Detection and Community Summarization in code.
- For community detection, we use the Leiden algorithm, known for good performance.
- In GraphRAG, we create an index called an artifact. We ultimately store the artifact using the `save_artifact` function.

In [12]:
# Below we define functions to save and load the final Graph Index artifacts.

import pickle
from pathlib import Path
import pandas as pd
from langchain_graphrag.indexing.artifacts import IndexerArtifacts


# This function saves the IndexerArtifacts object to disk.
def save_artifacts(artifacts: IndexerArtifacts, path: str):
    artifacts.entities.to_parquet(f"{path}/entities.parquet")
    artifacts.relationships.to_parquet(f"{path}/relationships.parquet")
    artifacts.text_units.to_parquet(f"{path}/text_units.parquet")
    artifacts.communities_reports.to_parquet(f"{path}/communities_reports.parquet")

    if artifacts.merged_graph is not None:
        with path.joinpath("merged-graph.pickle").open("wb") as fp:
            pickle.dump(artifacts.merged_graph, fp)

    if artifacts.summarized_graph is not None:
        with path.joinpath("summarized-graph.pickle").open("wb") as fp:
            pickle.dump(artifacts.summarized_graph, fp)

    if artifacts.communities is not None:
        with path.joinpath("community_info.pickle").open("wb") as fp:
            pickle.dump(artifacts.communities, fp)


# This function loads the IndexerArtifacts object from disk.
def load_artifacts(path: Path) -> IndexerArtifacts:
    entities = pd.read_parquet(f"{path}/entities.parquet")
    relationships = pd.read_parquet(f"{path}/relationships.parquet")
    text_units = pd.read_parquet(f"{path}/text_units.parquet")
    communities_reports = pd.read_parquet(f"{path}/communities_reports.parquet")

    merged_graph = None
    summarized_graph = None
    communities = None

    merged_graph_pickled = path.joinpath("merged-graph.pickle")
    if merged_graph_pickled.exists():
        with merged_graph_pickled.open("rb") as fp:
            merged_graph = pickle.load(fp)

    summarized_graph_pickled = path.joinpath("summarized-graph.pickle")
    if summarized_graph_pickled.exists():
        with summarized_graph_pickled.open("rb") as fp:
            summarized_graph = pickle.load(fp)

    community_info_pickled = path.joinpath("community_info.pickle")
    if community_info_pickled.exists():
        with community_info_pickled.open("rb") as fp:
            communities = pickle.load(fp)

    return IndexerArtifacts(
        entities,
        relationships,
        text_units,
        communities_reports,
        merged_graph=merged_graph,
        summarized_graph=summarized_graph,
        communities=communities,
    )

In [13]:
# The entire indexing pipeline: from chunking, graph generation, community detection to generating final artifacts (entities, relationships, communities, etc.).
# This step can take about 22 minutes.

from langchain_chroma.vectorstores import Chroma as ChromaVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_graphrag.indexing import SimpleIndexer
from langchain_graphrag.indexing.artifacts_generation import (
    CommunitiesReportsArtifactsGenerator,
    EntitiesArtifactsGenerator,
    RelationshipsArtifactsGenerator,
    TextUnitsArtifactsGenerator,
)
from langchain_graphrag.indexing.graph_clustering.leiden_community_detector import (
    HierarchicalLeidenCommunityDetector,
)
from langchain_graphrag.indexing.report_generation import (
    CommunityReportGenerator,
    CommunityReportWriter,
)

# Initialize the community detector
community_detector = HierarchicalLeidenCommunityDetector()

# Define the LLM and embedding model
ls_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create a Chroma vector store for entities.
entities_collection_name = f"entity-openai-embeddings"
entities_vector_store = ChromaVectorStore(
    collection_name=entities_collection_name,
    persist_directory="./",
    embedding_function=embeddings,
)

# Generators for different artifact types
entities_artifacts_generator = EntitiesArtifactsGenerator(
    entities_vector_store=entities_vector_store
)
relationships_artifacts_generator = RelationshipsArtifactsGenerator()

# Community Report Generator & Writer
report_generator = CommunityReportGenerator.build_default(
    llm=ls_llm,
    chain_config={"tags": ["community-report"]},
)
report_writer = CommunityReportWriter()

communities_report_artifacts_generator = CommunitiesReportsArtifactsGenerator(
    report_generator=report_generator,
    report_writer=report_writer,
)

# For text units
text_units_artifacts_generator = TextUnitsArtifactsGenerator()

# The SimpleIndexer orchestrates all indexing steps
indexer = SimpleIndexer(
    text_unit_extractor=text_unit_extractor,
    graph_generator=graph_generator,
    community_detector=community_detector,
    entities_artifacts_generator=entities_artifacts_generator,
    relationships_artifacts_generator=relationships_artifacts_generator,
    text_units_artifacts_generator=text_units_artifacts_generator,
    communities_report_artifacts_generator=communities_report_artifacts_generator,
)

# Run the entire pipeline on the loaded docs
artifacts = indexer.run(docs)

# Save the final artifacts to disk
artifacts_dir = Path("./")
save_artifacts(artifacts, artifacts_dir)

Generating report for level=2 commnuity_id=21: 100%|██████████| 2/2 [00:19<00:00,  9.57s/it]


## Local Search through Knowledge Graph

We perform a local search using the Knowledge Graph built by GraphRAG. Local Search is helpful for retrieving specific passages or details. Compare this with a simple gpt-4o-mini answer. The GraphRAG-based answer is much more detailed and grounded in the paper content.

In [14]:
# Load the artifacts we saved earlier
artifacts = load_artifacts(artifacts_dir)

In [15]:
# Now we demonstrate local search on the knowledge graph.

from typing import cast

from langchain_graphrag.query.local_search import (
    LocalSearch,
    LocalSearchPromptBuilder,
    LocalSearchRetriever,
)
from langchain_graphrag.query.local_search.context_builders import (
    ContextBuilder,
)
from langchain_graphrag.query.local_search.context_selectors import (
    ContextSelector,
)
from langchain_graphrag.types.graphs.community import CommunityLevel
from langchain_graphrag.utils import TiktokenCounter

# Build a default ContextSelector that will figure out which entities and communities
# are most relevant based on the user query.
context_selector = ContextSelector.build_default(
    entities_vector_store=entities_vector_store,
    entities_top_k=10,
    community_level=cast(CommunityLevel, 2),  # e.g. second-level community granularity
)

# The ContextBuilder merges the context from the selection step into a final prompt context.
context_builder = ContextBuilder.build_default(
    token_counter=TiktokenCounter(),
)

# LocalSearchRetriever uses the context_selector and context_builder to retrieve relevant data.
retriever = LocalSearchRetriever(
    context_selector=context_selector,
    context_builder=context_builder,
    artifacts=artifacts,
)

# LocalSearch ties everything together into a query chain.
local_search = LocalSearch(
    prompt_builder=LocalSearchPromptBuilder(),
    llm=ls_llm,
    retriever=retriever,
)

# We get a chain object by calling local_search().
search_chain = local_search()

# Let's make a query.
query = "What community detection algorithm does use in GraphRAG?"
print(search_chain.invoke(query))

## Community Detection in Graph RAG

Graph RAG employs the **Leiden algorithm** for community detection within its framework. The Leiden algorithm is recognized for its effectiveness in identifying well-connected communities in large networks, making it a suitable choice for analyzing complex datasets like the MultiHop-RAG dataset. This algorithm was developed by Traag et al. in 2019 and has since become a foundational tool in the field of community detection [Data: Entities (89); Relationships (132, 139)].

### Application in MultiHop-RAG

The MultiHop-RAG dataset, which includes a wide range of news articles published from September 2013 to December 2023, serves as a benchmark for evaluating various community detection algorithms, including the Leiden algorithm. The dataset is specifically designed to facilitate data sensemaking and explicit fact retrieval, showcasing the practical applications of the Leiden algorithm in real-world scenarios [Data: Entities (88); Relationships (135)]

## Global Search through Knowledge Graph

We can also perform a global search using the Knowledge Graph built by GraphRAG. A global search is useful for getting answers with broader context. However, global search requires a model with a sufficiently large max token length. For example, `gpt-4o-mini` (max token size = 16k) may not handle the entire content of a large paper-based graph. If possible, try `gpt-4o` (max token size = 32k) for larger contexts!

In [16]:
# Demonstrate global search using the knowledge graph.
# Here, we generate key points from the entire graph and then aggregate them.

from langchain_graphrag.query.global_search import GlobalSearch
from langchain_graphrag.query.global_search.community_weight_calculator import (
    CommunityWeightCalculator,
)
from langchain_graphrag.query.global_search.key_points_aggregator import (
    KeyPointsAggregator,
    KeyPointsAggregatorPromptBuilder,
    KeyPointsContextBuilder,
)
from langchain_graphrag.query.global_search.key_points_generator import (
    CommunityReportContextBuilder,
    KeyPointsGenerator,
    KeyPointsGeneratorPromptBuilder,
)

# The CommunityReportContextBuilder creates a context from community reports based on user query.
report_context_builder = CommunityReportContextBuilder(
    community_level=cast(CommunityLevel, 1),
    weight_calculator=CommunityWeightCalculator(),
    artifacts=artifacts,
    token_counter=TiktokenCounter(),
    max_tokens=16384,  # This is the limit for 'gpt-4o-mini'.
)

# KeyPointsGenerator creates key points from the available content.
kp_generator = KeyPointsGenerator(
    llm=ls_llm,
    prompt_builder=KeyPointsGeneratorPromptBuilder(
        show_references=True, repeat_instructions=True
    ),
    context_builder=report_context_builder,
)

# KeyPointsAggregator merges (aggregates) the key points from each relevant community.
kp_aggregator = KeyPointsAggregator(
    llm=ls_llm,
    prompt_builder=KeyPointsAggregatorPromptBuilder(
        show_references=True, repeat_instructions=True
    ),
    context_builder=KeyPointsContextBuilder(token_counter=TiktokenCounter()),
    output_raw=False,
)

# GlobalSearch orchestrates the generation and aggregation of key points for an entire knowledge graph.
global_search = GlobalSearch(
    kp_generator=kp_generator,
    kp_aggregator=kp_aggregator,
    generation_chain_config={"tags": ["kp-generation"]},
    aggregation_chain_config={"tags": ["kp-aggregation"]},
)

# Perform a synchronous invoke of the global search.
response = global_search.invoke(query)
print(response)

## Community Detection in GraphRAG

In the context of the Graph RAG, the community detection algorithm employed is the **Leiden algorithm**. This algorithm is particularly significant for analyzing the MultiHop-RAG dataset, as it is designed to effectively identify communities within large networks. The use of the Leiden algorithm highlights its importance in the research efforts surrounding community detection, showcasing its capability to enhance the understanding of network structures and relationships.

The application of the Leiden algorithm may lead to more accurate and meaningful insights into the community dynamics present in the dataset, which can be crucial for various analytical purposes. 

For further details, you may refer to the relevant data records [Data: Reports (8)].
